# TSP

In [26]:
# Encode the TSP using qlasskit and parameters

from qlasskit import qlassf, Parameter, Qlist, Qmatrix, Qint


@qlassf
def tsp(
    dst_matrix: Parameter[Qmatrix[Qint[4], 3, 3]], order: Qlist[Qint[2], 3]
) -> Qint[6]:
    dst_sum = Qint4(0)
    assertion = False
    
    if sum(order) != 3:
        assertion = True

    for i in range(2):
        oim = order[i]
        oi = order[i + 1]
        dst_sum += dst_matrix[oim][oi] if not assertion else 0xF
    return dst_sum


dst_matrix=[
    [0xF, 1, 4], 
    [1, 0xF, 1], 
    [1, 1, 0xF]
]
tsp_f = tsp.bind(dst_matrix=dst_matrix)
# tsp_f.expressions

In [27]:
bqm = tsp_f.to_bqm()
print("Vars:", bqm.num_variables, "\nInteractions:", bqm.num_interactions)

Vars: 18 
Interactions: 81


In [28]:
import neal
from qlasskit.bqm import decode_samples

sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=10)
decoded_samples = decode_samples(tsp_f, sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
print(best_sample.sample)


{'order': (0, 1, 2)}
